In [16]:
import faiss
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [17]:
name = 'mnist-784-euclidean'
nameFull = 'mnist-784-euclidean-true-labels.xlsx'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

<string>:57: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


trainDataset :  (60000, 784)
testDataset :  (10000, 784)


<string>:58: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
<string>:59: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


***Create faiss***

In [18]:
def createIndex(indexMethod, datasetImages):
    d = datasetImages.shape[1] # dimension
    M = 100
    time_start = perf_counter()
    index = indexMethod(faiss.IndexFlatL2(d), d, M, faiss.METRIC_L2)
    index.train(datasetImages)
    index.add(datasetImages) 
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    print(f'Took {totalTime:.3f} seconds')
    return (index, totalTime)
(indexedStruct, time) = createIndex(faiss.IndexIVFFlat, datasetTrainImages)

Took 1.610 seconds


In [19]:
print(indexedStruct)

<faiss.swigfaiss.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x000001B9E5A5EC60> >


In [20]:
# (min, max) = createIndexNumerous(createIndex, NearestNeighbors, datasetImages, 10)
# print('min : ', min, '\n','max : ', max,)

In [21]:
indexes = []
distances = []
def measureTime(par, indexes, distances, datasetImages):
    k=100
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        distance, index = indexedStruct.search(xq, k) 
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        distances.append(np.sqrt(distance[0]))
        indexes.append(index[0])
    # report the duration
    print(f'Took {totalTime:.3f} seconds')
measureTime(1000, indexes, distances, datasetTestImages)

Took 0.852 seconds


In [22]:
# (min, max) = measureTimeNumerous(measureTime, 10)
# print('min : ', min, '\n', 'max : ', max,)

In [23]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [24]:
print('indexes : ', indexes.shape)
print('distances : ', distances.shape)

indexes :  (1000, 100)
distances :  (1000, 100)


In [25]:
path = '../datasets/'+nameFull
(trueIndexes, trueDistances) = readDB(path)

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [26]:
compareFirstTen(indexes, distances, trueIndexes, trueDistances)

53843 || 53843
676.584 || 676.584
27059 || 38620
864.5039 || 793.9868
47003 || 16186
894.7 || 862.6766
44566 || 27059
917.6323 || 864.5039
15260 || 47003
921.6241 || 894.7
40368 || 14563
922.147 || 909.7043
36395 || 44566
943.4972 || 917.6323
30502 || 15260
954.8581 || 921.6241
17228 || 40368
1005.5889 || 922.147
35919 || 36395
1009.3607 || 943.4972


In [27]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)

0.0729
0.1588
0.8379
